# Example: using a different force field

Here we show how to use MDPOW to calculate partition coefficients using an force field that isn't included in the package. To follow along, install `jupyter` in your `mdpow` environment.

To implement a new force field, you will need:

* `ITP` files for the molecule, the solvents, ions and also the general atom type definitions file (usually just named after the force field itself).
* `MDP` files for the energy minimisation, initial relaxation, NPT ensemble run and free energy calculation.
* Structure files (`.gro` or `.pdb`) for the solute and non-aqueous solvent. If you are using a type of water that does not come bundled with GROMACS, like in this example, you will also need to create an equilibrated box of pure water.

The first thing we'll do is to download the files we need for Martini 3.0.

In [2]:
from pathlib import Path
from typing import Optional

import requests as req
from zipfile import ZipFile

HERE = Path(".")

MARTINI_ITP = HERE / "martini_v3.0.0.itp"
MARTINI_IONS = HERE / "martini_v3.0.0_ions_v1.itp"
MARTINI_SMALL_MOLS = HERE / "martini_v3.0.0_small_molecules_v1.itp"
MARTINI_SOLVENTS = HERE / "martini_v3.0.0_solvents_v1.itp"
MARTINI_WATER = HERE / "water.gro"
MARTINI_OCTANOL = HERE / "octanol.gro"

EM_FILE = HERE / "em.mdp"
EQ_FILE = HERE / "eq.mdp"
RUN_FILE = HERE / "run.mdp"

def download_file(
    url: str, out: Optional[Path] = None, chunk_size: int = 128, overwrite: bool = False
):
    """Utility function to download files."""
    if out is None:
        out = HERE / Path(url).name

    if out.exists() and not overwrite:
        return

    r = req.get(url, stream=True)
    r.raise_for_status()

    with out.open("wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)


ZIP_DOWNLOAD = {
    HERE / "BENZ.itp": "https://mad.ibcp.fr/api/molecule/download?id=731653322400583591&filename=BENZ.zip"
}
DOWNLOADS = {
    MARTINI_ITP: "https://github.com/marrink-lab/martini-forcefields/blob/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0.itp",
    MARTINI_IONS: "https://github.com/marrink-lab/martini-forcefields/blob/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_ions_v1.itp",
    MARTINI_SMALL_MOLS: "https://github.com/marrink-lab/martini-forcefields/blob/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_small_molecules_v1.itp",
    MARTINI_SOLVENTS: "https://github.com/marrink-lab/martini-forcefields/blob/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_solvents_v1.itp",
}
DOWNLOADS.update(ZIP_DOWNLOAD)

for fname, url in DOWNLOADS.items():
    download_file(url, fname)

for zip_file in ZIP_DOWNLOAD.keys():
    with ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(HERE)

This should have downloaded several files to your workspace.

In [ ]:
import gromacs
from mdpow import equil, forcefields, fep